In [1]:
!pip install mtcnn 

In [2]:
import cv2 as cv 
import os 
import numpy as np
import tensorflow as tf 
import matplotlib.pyplot as plt 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

2024-12-25 18:56:35.835421: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-25 18:56:35.946773: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-25 18:56:35.993811: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-25 18:56:36.006897: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-25 18:56:36.088450: I tensorflow/core/platform/cpu_feature_guar

In [3]:
img = cv.imread("dataset/person1/picture1.jpg")
#opencv read BGR channel format and plt reads images as RGB channel format 

[ WARN:0@2.904] global loadsave.cpp:248 findDecoder imread_('dataset/person1/picture1.jpg'): can't open/read file: check file path/integrity


In [4]:
from mtcnn.mtcnn import MTCNN

detector = MTCNN()
#mtcnn is RGB channel mode 
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
results = detector.detect_faces(img)

I0000 00:00:1735134999.077223    2135 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1735134999.233590    2135 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1735134999.233655    2135 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1735134999.237525    2135 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1735134999.237577    2135 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

InvalidImage: Image not valid.

In [ ]:
results

# For now we only need 'box' value here 

In [ ]:
x,y,w,h = results[0]['box']

In [ ]:
img = cv.rectangle(img, pt1=(x,y), pt2=(x+w, y+h), color=(0,0,255), thickness=5)
plt.imshow(img)

In [ ]:
my_face = img[y:y+h, x:x+w]
plt.imshow(my_face)

In [ ]:
#resize it for FaseNet bc it takes as input 160x160 size image 
my_face = cv.resize(my_face, (160,160))
plt.imshow(my_face)

In [ ]:
my_face

In [ ]:
print(tf.config.experimental.list_physical_devices('GPU'))

In [ ]:
# Now time to automate this process 
class FACELOADING:

    def __init__(self, directory: str) -> None:
        self.directory = directory
        self.target_size = (160, 160)
        self.X = [] # all images in X array 
        self.Y = [] # all labels in Y array 
        self.detector = MTCNN()

    # handle every image 
    def extract_face(self, filename: str) -> np.ndarray:
        img = cv.imread(filename)
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        x,y,w,h = detector.detect_faces(img)[0]['box']
        x, y = abs(x), abs(y) # for debugging purposes
        face = img[y:y+h, x:x+w]
        face_arr = cv.resize(face, self.target_size)
        return face_arr

    
    def load_faces(self, dir:str) -> list:
        FACES = []
        for im_name in os.listdir(dir):
            try:
                path = dir + im_name
                single_face = self.extract_face(filename=path)
                FACES.append(single_face)
            except Exception as e:
                pass
        return FACES


    def load_classes(self):
        for sub_dir in os.listdir(self.directory):
            path = self.directory + '/' + sub_dir + '/'
            FACES = self.load_faces(path)
            labels = [sub_dir for _ in range(len(FACES))]
            print(f"loaded successfully: {len(labels)}")
            self.X.extend(FACES) # here we use extend() method because append add all FACE list as a single element which is not we want
            self.Y.extend(labels)
        return np.asarray(self.X), np.asarray(self.Y)


    def plot_images(self):
        plt.figure(figsize=(16,12))
        for num, image in enumerate(self.X):
            ncols = 3 
            nrows = len(self.Y)//ncols + 2
            plt.subplot(nrows, ncols, num+1)
            plt.imshow(image)
            plt.axis("off")

In [ ]:
faceloading = FACELOADING("dataset")
X , Y = faceloading.load_classes()

In [ ]:
faceloading.plot_images()

In [ ]:
!pip install keras-facenet

In [ ]:
from keras_facenet import FaceNet

embedder = FaceNet()

def get_embedding(face_img):
    face_img = face_img.astype('float32') # 3D tensor (160 x 160 x 3)
    face_img = np.expand_dims(face_img, axis=0)  # tensorflow requires 4 dimensions (None x 160 x 160 x 3)
    yhat = embedder.embeddings(face_img)
    return yhat[0] # 512D image (1x1x512)
    yhat[0].ndim

In [ ]:
EMBEDDED_X = []

for img in X:
    EMBEDDED_X.append(get_embedding(img))

EMBEDDED_X = np.asarray(EMBEDDED_X)

In [ ]:
np.savez_compressed('faces.npz', EMBEDDED_X, Y)

## SVM model

Support Vector Machines (SVM) work on the principle of finding an optimal hyperplane that best separates the data points from different classes in a high-dimensional space.

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

encoder.fit(Y)
Y = encoder.transform(Y)

Y

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(EMBEDDED_X, Y, shuffle=True, random_state=42)

In [ ]:
from sklearn.svm import SVC

model = SVC(kernel='linear', probability=True)
# start with "linear kernel" and test on confusion matrix number of false positives, if SVM is underperforming switch kernel to "RBF" 
model.fit(X_train, Y_train)

In [ ]:
ypreds_train = model.predict(X_train)
ypreds_test = model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(Y_train, ypreds_train)

In [ ]:
accuracy_score(Y_test, ypreds_test)

In [ ]:
import pickle
with open("svm.pkl", 'wb') as file:
    pickle.dump(model, file)

In [ ]:
with open("svm.pkl", 'wb') as file:
    pickle.dump(model, file)